In [5]:
#imports
import datetime
import numpy as np
from pathlib import Path
import os
import time

import matplotlib
import matplotlib.pyplot as plt
    
from ocean_lib.example_config import ExampleConfig
from ocean_lib.ocean.ocean import Ocean
from ocean_lib.web3_internal.wallet import Wallet


#helper functions: setup
def create_ocean_instance() -> Ocean:
    config = ExampleConfig.get_config("https://polygon-rpc.com") # points to Polygon mainnet
    config["BLOCK_CONFIRMATIONS"] = 1 #faster
    ocean = Ocean(config)
    return ocean


def create_alice_wallet(ocean: Ocean) -> Wallet:
    config = ocean.config_dict
    alice_private_key = os.getenv('REMOTE_TEST_PRIVATE_KEY1')
    alice_wallet = Wallet(ocean.web3, alice_private_key, config["BLOCK_CONFIRMATIONS"], config["TRANSACTION_TIMEOUT"])
    bal = ocean.from_wei(alice_wallet.web3.eth.get_balance(alice_wallet.address))
    print(f"alice_wallet.address={alice_wallet.address}. bal={bal}")
    assert bal > 0, f"Alice needs MATIC"
    return alice_wallet


#helper functions: time
def to_unixtime(dt: datetime.datetime):
    return time.mktime(dt.timetuple())


def to_unixtimes(dts: list) -> list:
    return [to_unixtime(dt) for dt in dts]


def to_datetime(ut) -> datetime.datetime:
    return datetime.datetime.utcfromtimestamp(ut)


def to_datetimes(uts: list) -> list:
    return [to_datetime(ut) for ut in uts]


def round_to_nearest_hour(dt: datetime.datetime) -> datetime.datetime:
    return (dt.replace(second=0, microsecond=0, minute=0, hour=dt.hour)
            + datetime.timedelta(hours=dt.minute//30))


def pretty_time(dt: datetime.datetime) -> str:
    return dt.strftime('%Y/%m/%d, %H:%M:%S')


def print_datetime_info(descr:str, uts: list):
    dts = to_datetimes(uts)
    print(descr + ":")
    print(f"  starts on: {pretty_time(dts[0])}")
    print(f"    ends on: {pretty_time(dts[-1])}")
    print(f"  {len(dts)} datapoints")
    print(f"  time interval between datapoints: {(dts[1]-dts[0])}")


def target_12h_unixtimes(start_dt: datetime.datetime) -> list:
    target_dts = [start_dt + datetime.timedelta(hours=h) for h in range(12)]
    target_uts = to_unixtimes(target_dts)
    return target_uts


#helper-functions: higher level
def load_from_ohlc_data(file_name: str) -> tuple:
    """Returns (list_of_unixtimes, list_of_close_prices)"""
    with open(file_name, "r") as file:
        data_str = file.read().rstrip().replace('"', '')
    x = eval(data_str) #list of lists
    uts = [xi[0]/1000 for xi in x]
    vals = [xi[4] for xi in x]
    return (uts, vals)


def filter_to_target_uts(target_uts:list, unfiltered_uts:list, unfiltered_vals:list) -> list:
    """Return filtered_vals -- values at at the target timestamps"""
    filtered_vals = [None] * len(target_uts)
    for i, target_ut in enumerate(target_uts):
        time_diffs = np.abs(np.asarray(unfiltered_uts) - target_ut)
        tol_s = 1 #should always align within e.g. 1 second
        target_ut_s = pretty_time(to_datetime(target_ut))
        assert min(time_diffs) <= tol_s, \
            f"Unfiltered times is missing target time: {target_ut_s}"
        j = np.argmin(time_diffs)
        filtered_vals[i] = unfiltered_vals[j]
    return filtered_vals


#helpers: save/load list
def save_list(list_: list, file_name: str):
    """Save a file shaped: [1.2, 3.4, 5.6, ..]"""
    p = Path(file_name)
    p.write_text(str(list_))


def load_list(file_name: str) -> list:
    """Load from a file shaped: [1.2, 3.4, 5.6, ..]"""
    p = Path(file_name)
    s = p.read_text()
    list_ = eval(s)
    return list_


#helpers: prediction performance
def calc_nmse(y, yhat) -> float:
    assert len(y) == len(yhat)
    mse_xy = np.sum(np.square(np.asarray(y) - np.asarray(yhat)))
    mse_x = np.sum(np.square(np.asarray(y)))
    nmse = mse_xy / mse_x
    return nmse


def plot_prices(cex_vals, pred_vals):
    matplotlib.rcParams.update({'font.size': 22})
    
    x = [h for h in range(0,12)]
    assert len(x) == len(cex_vals) == len(pred_vals)
    
    fig, ax = plt.subplots()
    ax.plot(x, cex_vals, '--', label="CEX values")
    ax.plot(x, pred_vals, '-', label="Pred. values")
    ax.legend(loc='lower right')
    plt.ylabel("ETH price")
    plt.xlabel("Hour")
    fig.set_size_inches(18, 18)
    plt.xticks(x)
    plt.show()


In [4]:
os.environ["REMOTE_TEST_PRIVATE_KEY1"] = "c418c07d9291f16f8ae12b8121cff4207ea873c9f5dd0908ee6c2bdbd13de6b0"
ocean = create_ocean_instance()
alice_wallet = create_alice_wallet(ocean)

alice_wallet.address=0x19081dEd691eE3Cb1A45B19ae68Dd8d2A911b2e3. bal=4.924948875111925708


In [7]:
start_dt = datetime.datetime.now() - datetime.timedelta(hours=24) #must be >= 12h ago
start_dt = round_to_nearest_hour(start_dt) # so that times line up
target_uts = target_12h_unixtimes(start_dt)
print_datetime_info("target times", target_uts)

import ccxt
allcex_x = ccxt.binance().fetch_ohlcv('ETH/USDT', '1h')
allcex_uts = [xi[0]/1000 for xi in allcex_x]
allcex_vals = [xi[4] for xi in allcex_x]
print_datetime_info("allcex times", allcex_uts)

cex_vals = filter_to_target_uts(target_uts, allcex_uts, allcex_vals)

cex_vals

target times:
  starts on: 2022/11/04, 15:00:00
    ends on: 2022/11/05, 02:00:00
  12 datapoints
  time interval between datapoints: 1:00:00
allcex times:
  starts on: 2022/10/15, 20:00:00
    ends on: 2022/11/05, 15:00:00
  500 datapoints
  time interval between datapoints: 1:00:00


[1624.97,
 1621.9,
 1622.83,
 1637.51,
 1648.29,
 1645.55,
 1648.37,
 1649.66,
 1644.78,
 1658.28,
 1653.72,
 1648.97]

In [ ]:
nmse = calc_nmse(cex_vals, pred_vals)
print(f"NMSE = {nmse}")
plot_prices(cex_vals, pred_vals)

In [7]:
import pandas as pd
df = pd.DataFrame({'x':[1,2,3,4]})
df.to_csv('test.csv')

In [9]:
from pybundlr import pybundlr
file_name = "test.csv"
url = pybundlr.fund_and_upload(file_name, "matic", alice_wallet.private_key)
#e.g. url = "https://arweave.net/qctEbPb3CjvU8LmV3G_mynX74eCxo1domFQIlOBH1xU"
print(f"Your csv url: {url}")


RUN COMMAND: bundlr price 19 -c matic -h https://node1.bundlr.network 
Price for 19 bytes in matic is 45240962932936 wei (0.000045240962932936 matic)


RUN COMMAND: bundlr balance 0x19081dEd691eE3Cb1A45B19ae68Dd8d2A911b2e3 -c matic -h https://node1.bundlr.network
Balance: 0 wei (0 matic)


RUN COMMAND: bundlr fund 90481925865872 -c matic -h https://node1.bundlr.network -w c418c0... --no-confirmation
Loaded address: 0x19081ded691ee3cb1a45b19ae68dd8d2a911b2e3
Funding receipt: 
Amount: 90481925865872 with Fee: "1258230441930000" to 0xb4DE0833771eae55040b698aF5eB06d59E142C82 
Transaction ID: 0x6945a080161e93a68ddd56177c298d16a63f0595d51614af667509a637e8d653 


RUN COMMAND: bundlr upload test.csv -c matic -h https://node1.bundlr.network -w c418c0... --no-confirmation
Loaded address: 0x19081ded691ee3cb1a45b19ae68dd8d2a911b2e3
Uploaded to https://arweave.net/raXN4s7KTvPWelcHyVsb2AGBgTlCS5Ro4X_Vajs0twA

Your csv url: https://arweave.net/raXN4s7KTvPWelcHyVsb2AGBgTlCS5Ro4X_Vajs0twA


In [10]:
name = "Test CSV Data OMGSquad " + str(time.time()) #time for unique name
(data_nft, datatoken, asset) = ocean.assets.create_url_asset(name, url, alice_wallet, wait_for_aqua=False)
data_nft.set_metadata_state(metadata_state=5, from_wallet=alice_wallet)
print(f"New asset created, with did={asset.did}, and datatoken.address={datatoken.address}")

INFO:ocean:Successfully created NFT with address 0xD0003D2E38A7f3c5b44B6c735b7CFC612b76849A.
INFO:ocean:Successfully created datatoken with address 0xC1E2A2E9a64F9f939Ac8Ee832F3126DC8FfFc2e0.
INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x04d4be89435e368518e608d0232f9f27f8c5ed1d84f1df945cdb9db7175857e1eb43482b2e1620452b9324f367f95aab94f27a3c433cc36fc9f65805b7f01773126f758c42ac6c6b7879adc795565814fa47bc86e3625addaca41be0af1b4eb560ff7cbd037991539e82b5f11c99505cb2ac282199a5ea989af6899abefc818db26f70370b6fc7c51b5fe08cf3ebb5f28be402a129555dfeac656e3359ca5251e79eedc8e095a6303af18701006620f225db75dc736d71c014b01feaedecdb665dbb1e6faf6f4ac4eae595b1a9b06fc2c9b73a171b07117eeff0c48727132dd81d423fbcccf5f8d6d4a208532717c4b0c17c21827d56b7cd82fbc7e253f9a7e200f771fa4b13f2a4cd1b96f86c1747cf3972e1e199bd9c89b27751618342bdaa3cfb83be907bed91d66755c685ab9c46cf2757f693443ec87f289ff578, encryptedEndpoint https://v4.provider.polygon.oceanprotocol.com/api/ser

New asset created, with did=did:op:89e955c20de109b33ab5a2ddc30e80cd24123ad46257d2866f5f168703237fac, and datatoken.address=0xC1E2A2E9a64F9f939Ac8Ee832F3126DC8FfFc2e0
